# **Kaggle – DataTops®**

Tu TA ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Aspectos importantes

- Última submission:
	- Mañana: 17 de febrero a las 5pm
	- Tarde: 19 de febrero a las 5pm
- **Enlace de la competición**: https://www.kaggle.com/t/c5cc87b50c4b4770bdc8f5acbe15577d
- **Requisito**: Estar registrado en [Kaggle](https://www.kaggle.com/)

## Métrica

El error cuadrático medio (RMSE, por sus siglas en inglés) es una medida de la desviación estándar de los residuos (errores de predicción). Los residuos representan la diferencia entre los valores observados y los valores predichos por el modelo. El RMSE indica qué tan dispersos están estos errores: cuanto menor es el RMSE, más cercanas están las predicciones a los valores reales. En otras palabras, el RMSE mide qué tan bien se ajusta la línea de regresión a los datos.

$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$

## 1. Librerías

In [1]:
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import urllib.request

## 2. Datos

In [2]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle
df = pd.read_csv("./data/train.csv")
df.shape

(912, 13)

### 2.1 Exploración de los datos

In [23]:
def get_quality(dataframe: pd.DataFrame):
	if dataframe is None:
		raise ValueError("Dataframe sin especificar.")

	if not isinstance(dataframe, pd.DataFrame):
		raise ValueError("El dataframe debe ser pd.DataFrame")

	records = dataframe.shape[0]

	unique = dataframe.nunique().rename("Cardinality")
	cardinality = (unique/records).rename("Uniqueness")
	null = (dataframe.isna().sum()).rename("Null")

	data = pd.DataFrame(data=[unique, cardinality, null])
	return data.T

In [25]:
get_quality(df)

,Cardinality,Uniqueness,Null
laptop_ID,912.0,1.000000,0.0
Company,19.0,0.020833,0.0
Product,480.0,0.526316,0.0
TypeName,6.0,0.006579,0.0
Inches,17.0,0.018640,0.0
ScreenResolution,36.0,0.039474,0.0
Cpu,107.0,0.117325,0.0
Ram,9.0,0.009868,0.0
Memory,37.0,0.040570,0.0
Gpu,93.0,0.101974,0.0


In [5]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    str    
 2   Product           912 non-null    str    
 3   TypeName          912 non-null    str    
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    str    
 6   Cpu               912 non-null    str    
 7   Ram               912 non-null    str    
 8   Memory            912 non-null    str    
 9   Gpu               912 non-null    str    
 10  OpSys             912 non-null    str    
 11  Weight            912 non-null    str    
 12  Price_in_euros    912 non-null    float64
dtypes: float64(2), int64(1), str(10)
memory usage: 92.8 KB


In [6]:
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [7]:
df.tail()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
907,28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
908,1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
909,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
910,23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
911,229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [8]:
df.describe()

,laptop_ID,Inches,Price_in_euros
count,912.000000,912.000000,912.000000
mean,650.312500,14.981579,1111.724090
std,382.727748,1.436719,687.959172
min,2.000000,10.100000,174.000000
25%,324.750000,14.000000,589.000000
50%,636.500000,15.600000,978.000000
75%,982.250000,15.600000,1483.942500
max,1320.000000,18.400000,6099.000000


#### Transformaciones

##### `ScreenResolution`

In [3]:
import re

def extra_screen_data(value):
	touchscreen_match = re.search(r"Touchscreen", value, re.IGNORECASE)
	panel_match = re.search(r"(IPS Panel|LCD|OLED)", value, re.IGNORECASE)
	type_match = re.search(r"(Quad|Full|Ultra)\s?HD\+?", value, re.IGNORECASE)
	resolution_match = re.search(r"\d{3,4}x\d{3,4}", value, re.IGNORECASE)

	return {
		"has_touchscreen": True if touchscreen_match else False,
		"screen_panel": panel_match.group(0) if panel_match else None,
		"screen_type": type_match.group(0) if type_match else None,
		"screen_resolution": resolution_match.group(0) if resolution_match else None
	}

screen_data = df["ScreenResolution"].apply(extra_screen_data)
df = pd.concat([df, pd.DataFrame(screen_data.tolist())], axis=1)
df.sample(5).iloc[:, -5:]

,Price_in_euros,has_touchscreen,screen_panel,screen_type,screen_resolution
737,344.99,False,NaN,Full HD,1920x1080
297,629.00,False,NaN,NaN,1366x768
884,1199.00,False,NaN,Full HD,1920x1080
395,419.00,False,NaN,NaN,1366x768
724,1599.00,True,IPS Panel,Ultra HD,3840x2160


In [90]:
df.iloc[:, -5:]

,Price_in_euros,has_touchscreen,screen_panel,screen_type,screen_resolution
0,539.00,False,NaN,Full HD,1920x1080
1,879.01,False,NaN,Full HD,1920x1080
2,900.00,False,NaN,Full HD,1920x1080
3,898.94,False,NaN,NaN,1440x900
4,428.00,False,NaN,Full HD,1920x1080
...,...,...,...,...,...
907,800.00,False,NaN,Full HD,1920x1080
908,1629.00,True,NaN,Full HD,1920x1080
909,519.00,False,NaN,Full HD,1920x1080
910,258.00,False,NaN,NaN,1366x768


##### `CPU`

In [23]:
# ?P<family>Intel\s(Core|Celeron|Xeon|Atom|Pentium))
# ?P<series>((e|M|i)\d|(Single|Dual|Quad)\sCore)
# ?P<gen>(\d{1}(?=\d{3}))|(\d{2}(?=\d{3})))
# ?P<cpu>\d\.?\d?\s*GHz

# ?P<family>AMD\s
# ?P<model>(Ryzen|Athlon|Threadripper|EPYC)
mask = df["Cpu"].str.contains("Intel", True)
df.loc[:, "Cpu"].sample(30)

245              Intel Core i7 7500U 2.7GHz
303              Intel Core i5 8250U 1.6GHz
843    Intel Celeron Dual Core N3350 1.1GHz
239             Intel Core i7 6700HQ 2.6GHz
577              Intel Core i7 6500U 2.5GHz
580              Intel Core i7 7500U 2.7GHz
407              Intel Core i7 7500U 2.7GHz
582              Intel Core i3 7100U 2.4GHz
555              Intel Core i5 8250U 1.6GHz
825             Intel Core i7 7700HQ 2.8GHz
830                Intel Core i3 6006U 2GHz
644             Intel Core i7 7700HQ 2.8GHz
768              Intel Core i5 7200U 2.5GHz
776              Intel Core i3 6006U 2.0GHz
15     Intel Celeron Quad Core N3710 1.6GHz
810              Intel Core i7 8550U 1.8GHz
493              Intel Core i7 6500U 2.5GHz
75              Intel Atom x5-Z8550 1.44GHz
396              Intel Core i5 6200U 2.3GHz
736              Intel Core i5 7200U 2.5GHz
799              Intel Core i7 7660U 2.5GHz
405              Intel Core i7 6500U 2.5GHz
780              Intel Core i5 6

In [ ]:
"(?P<brand>AMD|INTEL)\s(?P<family>((?<=Intel\s)Core|Pentium|Atom|Celeron|Xeon)|(?<=AMD\s)([AE]\d{,2}-Series|FX|Ryzen))\s(?P<serie>((?<=Pentium\s|Celeron\s)(Dual|Single|Quad)\s?Core)|([ix]\d{,2})|([AE]\d{,2})|(?<=.\d-Series))?\s?(?P<gen>(?:-|A-Z|\s)?\d(?=\d{,4}))"gmi

Intel Core i7 7500U 2.7GHz
Intel Core i5 8250U 1.6GHz
Intel Celeron Dual Core N3350 1.1GHz
Intel Core i7 6700HQ 2.6GHz
Intel Core i7 6500U 2.5GHz
Intel Core i7 7500U 2.7GHz
Intel Core i7 7500U 2.7GHz
Intel Core i3 7100U 2.4GHz
Intel Core i5 8250U 1.6GHz
Intel Core i7 7700HQ 2.8GHz
Intel Core i3 6006U 2GHz
Intel Core i7 7700HQ 2.8GHz
Intel Core i5 7200U 2.5GHz
Intel Core i3 6006U 2.0GHz
Intel Celeron Quad Core N3710 1.6GHz
Intel Core i7 8550U 1.8GHz
Intel Core i7 6500U 2.5GHz
Intel Atom x5-Z8550 1.44GHz
Intel Core i5 6200U 2.3GHz
Intel Core i5 7200U 2.5GHz
Intel Core i7 7660U 2.5GHz
Intel Core i7 6500U 2.5GHz
Intel Core i5 6200U 2.3GHz
Intel Core i3 6006U 2GHz
Intel Core i5 7200U 2.5GHz
Intel Core i5 7200U 2.5GHz
Intel Celeron Dual Core N3050 1.6GHz
Intel Core i7 7500U 2.7GHz
Intel Core i5 7300HQ 2.5GHz
AMD A9-Series 9410 2.9GHz
Intel Core i7 8550U 1.8GHz
Intel Core i5 7200U 2.5GHz
Intel Core i7 7700HQ 2.8GHz
Intel Core i7 7500U 2.7GHz
Intel Core i5 8250U 1.6GHz
Intel Core i7 8550U 1.8GHz
Intel Pentium Dual Core 4405U 2.1GHz
Intel Core i3 6006U 2.2GHz
Intel Atom Z8350 1.92GHz
Intel Core i5 7200U 2.50GHz
Intel Pentium Dual Core N4200 1.1GHz
AMD A9-Series 9420 3GHz
AMD A6-Series 9220 2.5GHz
AMD A12-Series 9720P 3.6GHz
AMD Ryzen 1700 3GHz
AMD A9-Series A9-9420 3GHz
AMD A9-Series 9410 2.9GHz
AMD E-Series E2-9000e 1.5GHz
AMD A8-Series 7410 2.2GHz
AMD A6-Series A6-9220 2.5GHz
AMD A10-Series A10-9620P 2.5GHz
AMD FX 9830P 3GHz
AMD A6-Series 9220 2.9GHz
AMD E-Series 7110 1.8GHz
AMD E-Series 6110 1.5GHz
AMD E-Series 9000e 1.5GHz
AMD E-Series E2-6110 1.5GHz
AMD A10-Series 9600P 2.4GHz
AMD E-Series E2-9000 2.2GHz
AMD A9-Series 9420 2.9GHz
AMD A12-Series 9720P 2.7GHz
AMD A10-Series 9620P 2.5GHz
AMD Ryzen 1600 3.2GHz
AMD A6-Series 7310 2GHz

In [ ]:
# intel 
#(?P<family>Intel\s(Core|Celeron|Xeon|Atom|Pentium))|(i\d|(Dual|Quad)\sCore)|(?P<gen>(\d{1}(?=\d{3}))|(\d{2}(?=\d{3})))|(?P<cpu>\d\.?\d?\s*GHz)

def extra_screen_data(value):
	touchscreen_match = re.search(r"Touchscreen", value, re.IGNORECASE)
	panel_match = re.search(r"(IPS Panel|LCD|OLED)", value, re.IGNORECASE)
	type_match = re.search(r"(Quad|Full|Ultra)\s?HD\+?", value, re.IGNORECASE)
	resolution_match = re.search(r"\d{3,4}x\d{3,4}", value, re.IGNORECASE)

	return {
		"has_touchscreen": True if touchscreen_match else False,
		"screen_panel": panel_match.group(0) if panel_match else None,
		"screen_type": type_match.group(0) if type_match else None,
		"screen_resolution": resolution_match.group(0) if resolution_match else None
	}

screen_data = df["ScreenResolution"].apply(extra_screen_data)
df = pd.concat([df, pd.DataFrame(screen_data.tolist())], axis=1)
df.sample(25).iloc[:, -5:]

,screen_resolution,has_touchscreen,screen_panel,screen_type,screen_resolution
881,1920x1080,True,IPS Panel,Full HD,1920x1080
15,1366x768,False,NaN,NaN,1366x768
739,1366x768,False,NaN,NaN,1366x768
602,1366x768,False,NaN,NaN,1366x768
249,1920x1080,False,NaN,Full HD,1920x1080
799,2256x1504,True,NaN,NaN,2256x1504
518,1366x768,False,NaN,NaN,1366x768
604,1920x1080,False,IPS Panel,Full HD,1920x1080
536,1920x1080,False,NaN,Full HD,1920x1080
21,1920x1080,True,NaN,Full HD,1920x1080


### 2.3 Definir X e y

In [9]:
X = df.drop(["Price_in_euros"], axis=1)
y = df["Price_in_euros"].copy()

X.shape

(912, 12)

In [10]:
y.shape

(912,)

### 2.4 Dividir X_train, X_test, y_train, y_test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
	X,
	y,
	test_size=.20,
	random_state=42
)

In [12]:
X_train

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
25,1118,HP,ZBook 17,Workstation,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,AMD FirePro W6150M,Windows 7,3.0kg
84,153,Dell,Inspiron 5577,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1050,Windows 10,2.56kg
10,275,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.9GHz,8GB,512GB SSD,Intel Iris Graphics 550,macOS,1.37kg
342,1100,HP,EliteBook 840,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,500GB HDD,Intel HD Graphics 520,Windows 7,1.54kg
890,131,Dell,Inspiron 5770,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,256GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.8kg
...,...,...,...,...,...,...,...,...,...,...,...,...
106,578,HP,14-am079na (N3710/8GB/2TB/W10),Notebook,14.0,1366x768,Intel Pentium Quad Core N3710 1.6GHz,8GB,2TB HDD,Intel HD Graphics 405,Windows 10,1.94kg
270,996,Lenovo,IdeaPad 320-15ABR,Notebook,15.6,Full HD 1920x1080,AMD A12-Series 9720P 3.6GHz,6GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg
860,770,Dell,Latitude 7280,Ultrabook,12.5,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,16GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.18kg
435,407,Lenovo,IdeaPad 320-15IAP,Notebook,15.6,1366x768,Intel Celeron Dual Core N3350 1.1GHz,4GB,1TB HDD,Intel HD Graphics 500,Windows 10,2.2kg


In [13]:
y_train

25     2899.00
84     1249.26
10     1958.90
342    1030.99
890    1396.00
        ...   
106     389.00
270     549.00
860    1859.00
435     306.00
102    1943.00
Name: Price_in_euros, Length: 729, dtype: float64

## 3. Procesado de datos

Nuestro target es la columna `Price_in_euros`

In [14]:
def get_quality(dataframe: pd.DataFrame):
	if dataframe is None:
		raise ValueError("Dataframe sin especificar.")

	if not isinstance(dataframe, pd.DataFrame):
		raise ValueError("El dataframe debe ser pd.DataFrame")

	records = dataframe.shape[0]

	unique = dataframe.nunique().rename("Cardinality")
	cardinality = (unique/records).rename("Uniqueness")
	null = (dataframe.isna().sum()).rename("Null")

	data = pd.DataFrame(data=[unique, cardinality, null])
	return data.T

In [16]:
get_quality(X_train)

,Cardinality,Uniqueness,Null
laptop_ID,729.0,1.000000,0.0
Company,17.0,0.023320,0.0
Product,408.0,0.559671,0.0
TypeName,6.0,0.008230,0.0
Inches,15.0,0.020576,0.0
ScreenResolution,33.0,0.045267,0.0
Cpu,93.0,0.127572,0.0
Ram,8.0,0.010974,0.0
Memory,33.0,0.045267,0.0
Gpu,84.0,0.115226,0.0


In [22]:
y_train.describe()

count     729.000000
mean     1103.789314
std       672.778174
min       191.900000
25%       598.000000
50%       985.000000
75%      1460.000000
max      6099.000000
Name: Price_in_euros, dtype: float64

In [21]:
y_train.isnull().value_counts()

Price_in_euros
False    729
Name: count, dtype: int64

-----------------------------------------------------------------------------------------------------------------

## 4. Modelado

### 4.1 Baseline de modelos

### 4.2 Sacar métricas, valorar los modelos

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

### 4.3 Optimización (up to you 🫰🏻)

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

## 1. Carga los datos de `test.csv` para predecir.


In [ ]:
X_pred = pd.read_csv("./data/test.csv")
X_pred.head()

In [ ]:
X_pred.tail()

In [ ]:
X_pred.info()

 ## 2. Replicar el procesado para ``test.csv``

In [ ]:
X_pred

In [ ]:
predictions_submit = model.predict(X_pred)
predictions_submit

**¡OJO! ¿Por qué me da error?**

IMPORTANTE:

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO **BORRAR FILAS**, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER **391 FILAS**, SI O SI

**Entonces, si al cargar los datos de ``train.csv`` usaste `index_col=0`, ¿tendré que hacer lo también para el `test.csv`?**

In [ ]:
# ¿Qué opináis?
# ¿Sí, no?

![wow.jpeg](attachment:wow.jpeg)

## 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`.

In [ ]:
sample = pd.read_csv("data/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
sample.shape

## 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [ ]:
#¿Cómo creamos la submission?
submission = pd.DataFrame()

In [ ]:
submission.head()

In [ ]:
submission.shape

## 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.

    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.

    Si no, LEE EL MENSAJE Y HAZLE CASO.

    Si aún no:
    - apaga tu ordenador,
    - date una vuelta,
    - enciendelo otra vez,
    - abre este notebook y
    - leelo todo de nuevo.
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")
                img = Image.open("gfg.png")
                img.show()
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [ ]:
chequeador(submission)